In [115]:
import pandas as pd
import numpy as np
import os
import xlrd

# Profile 
from pandas_profiling import ProfileReport

import plotly.graph_objs as go

# Config
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)